# Server Test
> This is the code I am using to test the server. In this test, I will use 2 connections and attempt to make the join the same game.

In [1]:
import socket as s

## Connecting to server

I connect to the GO server on port 80(configured in docker).

In [2]:
conn = s.socket(s.AF_INET, s.SOCK_STREAM)
conn.connect(("127.0.0.1", 80))

## Sending packets

All packets must be suffixed with \n. This is a python implemtation of how to send them:

In [3]:
# Sends a packet, returns whatever socket.Send returns
def send_packet(content: bytes):
    global conn
    return conn.send(content + b'\n')


## Get a name

In [4]:
send_packet(b'njim')#set my name as jim
conn.recv(1024)

b'Njim\n'

## Creating a game

I need to send a packet with 'c' as the first character to intialise a game:

In [5]:
send_packet(b'c')

2

In response, I should get a packet with the *game code*.

In [6]:
BUFFER_SIZE = 1024

def recieve_packet() -> bytes:
    return conn.recv(BUFFER_SIZE)

## Recieve the response from creating the game

In [7]:
packet = recieve_packet()

In [8]:
trimmed = packet[1:-1]#trim type and \n out
import json
trimmed

b'qobeaqsn'

## User 2

I will attempt to join the game as another user

In [9]:
conn2 = s.socket(s.AF_INET, s.SOCK_STREAM)
conn2.connect(("127.0.0.1", 80))

In [10]:
conn2.send(b'nbob\n')#set my name as bob
conn2.recv(BUFFER_SIZE)

b'Nbob\n'

In [11]:
conn2.send(b'j' + trimmed + b'\n')

from time import sleep
sleep(1)#wait for response

conn2.recv(BUFFER_SIZE)

b'BS\nJsuccess\n'

### Conn 1 start game packet


In [12]:
conn.recv(BUFFER_SIZE)

b'BS\n'


## Playing the game
The two players should be able to share data with each other.

In [13]:
conn.send(
    b'f{"X":1, "Y":1, "Dx":999, "Dy":999}\n'
)

36

In [14]:
conn.recv(BUFFER_SIZE)

b'F{"Players":[{"Name":"jim","X":1,"Y":1,"Dx":999,"Dy":999,"Score":0},{"Name":"bob","X":0,"Y":0,"Dx":0,"Dy":0,"Score":0}],"State":1,"Name":"qobeaqsn"}\n'

subequent reads should have different values for player 1, becuase Dx and Dy move the player along like so

In [15]:
import time
time.sleep(1)

conn2.send(b'f{"X":1, "Y":1, "Dx":0, "Dy":0}\n')


32

In [16]:
conn2.recv(BUFFER_SIZE)

b'F{"Players":[{"Name":"jim","X":1199.8000000000004,"Y":1199.8000000000004,"Dx":999,"Dy":999,"Score":0},{"Name":"bob","X":1,"Y":1,"Dx":0,"Dy":0,"Score":0}],"State":1,"Name":"qobeaqsn"}\n'

## Error cases

In [17]:
conn3 = s.socket(s.AF_INET, s.SOCK_STREAM)
conn3.connect(("127.0.0.1", 80))

### Joining a game without a name

In [18]:
conn3.send(b'j{"Name":"' + trimmed + b'"}\n')
conn3.recv(BUFFER_SIZE)

b'ECannot join a game when you do not have a name.\n'

resolve this issue like so:

In [19]:
conn3.send(b'nJoe\n')
conn3.recv(BUFFER_SIZE)

b'NJoe\n'

### Joining a full/running game 
This should fail and return an error packet explaining why

In [20]:
conn3.send(b'j' + trimmed + b'\n')
conn3.recv(BUFFER_SIZE)

b'Ecould not join game\n'

### Creating a game with 1 player and attempting to send a frame
This should fail and return an error packet explaining why

In [21]:
conn3.send(b'c\n')
conn3.recv(BUFFER_SIZE)#i dont care what the game id is

conn3.send(
    b'f{"X":1, "Y":1, "Dx":0, "Dy":0}\n'
)
conn3.recv(BUFFER_SIZE)

b'EGame not started yet\n'

### Wrong names

In [22]:
conn4 = s.socket(s.AF_INET, s.SOCK_STREAM)
conn4.connect(("127.0.0.1", 80))

#### Not submitting a name
will return error

In [23]:
conn4.send(b"n\n")
conn4.recv(BUFFER_SIZE)

b'EName was either not long enough, too long, or contained banned characters.\n'

#### Submitting a name that is too long
This will also return an error explaining why

In [24]:
conn4.send(b"nmy obnoxiously massive name I chose to play as which does not make the game look fun and also could pose a risk of exploits for the server; DROP *;\n")
conn4.recv(BUFFER_SIZE)

b'EName was either not long enough, too long, or contained banned characters.\n'

#### All talk?
I try to create a game after all this to catch any bug that may have slipped through

In [25]:
conn4.send(b"c\n")
conn4.recv(BUFFER_SIZE)

b'ECannot create a game without having a name.\n'

## The target broadcasts
These should arrive on average every 5 seconds, so this is called after everyting with a 10s delay

In [26]:
sleep(10)

In [27]:
conn.recv(BUFFER_SIZE)

b'BT{"X":-4.49868513315349,"Y":-31.545469589927656,"Dx":-7.383619563057301,"Dy":-24.228390291508422,"ID":0}\n'

In [28]:
conn2.recv(BUFFER_SIZE)

b'BT{"X":-4.49868513315349,"Y":-31.545469589927656,"Dx":-7.383619563057301,"Dy":-24.228390291508422,"ID":0}\n'